## Downloading data from kaggle

In [2]:
!pip install -q kaggle

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"rajdiv","key":"8c9ce6c1290cd031b921daf021d79ffa"}'}

In [4]:
! mkdir ~/.kaggle

In [5]:
! cp kaggle.json ~/.kaggle/

In [6]:
! chmod 600 ~/.kaggle/kaggle.json

In [7]:
! kaggle datasets download -d brahma0545/aaic-assignment-tl

100% 4.34G/4.34G [00:40<00:00, 200MB/s]
100% 4.34G/4.34G [00:40<00:00, 116MB/s]


In [8]:
!unzip /content/aaic-assignment-tl.zip # Unzipping data

Streaming output truncated to the last 5000 lines.
  inflating: data_final/imagesx/x/j/f/xjf38e00/1003080851.tif  
  inflating: data_final/imagesx/x/j/g/xjg09d00/50434737-4738.tif  
  inflating: data_final/imagesx/x/j/g/xjg47c00/2082461891_1894.tif  
  inflating: data_final/imagesx/x/j/g/xjg64e00/1001764469_1001764471.tif  
  inflating: data_final/imagesx/x/j/g/xjg80d00/517546706+-6707.tif  
  inflating: data_final/imagesx/x/j/h/xjh09d00/50436353-6353.tif  
  inflating: data_final/imagesx/x/j/h/xjh28c00/507173521+-3522.tif  
  inflating: data_final/imagesx/x/j/h/xjh66d00/508560943+-0943.tif  
  inflating: data_final/imagesx/x/j/i/xji52d00/2021354149.tif  
  inflating: data_final/imagesx/x/j/j/xjj15c00/2084564898.tif  
  inflating: data_final/imagesx/x/j/k/xjk09e00/2054608885.tif  
  inflating: data_final/imagesx/x/j/k/xjk30e00/88180487.tif  
  inflating: data_final/imagesx/x/j/k/xjk49d00/501351704_501351706.tif  
  inflating: data_final/imagesx/x/j/k/xjk70a00/0060142584.tif  
  inflati

In [9]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras.applications.vgg16 import VGG16
from tensorflow import keras
from keras.models import Model
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, Input


In [10]:
import os 
os.getcwd()

'/content'

## Reading dataframe

In [11]:
df = pd.read_csv(r'/content/labels_final.csv',dtype=str)

In [12]:
df.head()

,path,label
0,imagesv/v/o/h/voh71d00/509132755+-2755.tif,3
1,imagesl/l/x/t/lxt19d00/502213303.tif,3
2,imagesx/x/e/d/xed05a00/2075325674.tif,2
3,imageso/o/j/b/ojb60d00/517511301+-1301.tif,3
4,imagesq/q/z/k/qzk17e00/2031320195.tif,7


## Using Image Data Generator

In [13]:
data_gen = ImageDataGenerator(rescale=1/255., validation_split=0.25)

In [14]:
colnames=['path', 'label'] 
train_data = data_gen.flow_from_dataframe(dataframe=df, directory="/content/data_final", x_col=colnames[0], y_col=colnames[1], 
                                          class_mode="categorical", target_size=(256,256), batch_size=32, subset='training', seed = 9)

Found 36000 validated image filenames belonging to 16 classes.


In [15]:
validation_data = data_gen.flow_from_dataframe(dataframe=df, directory="/content/data_final", x_col=colnames[0], y_col=colnames[1], 
                                          class_mode="categorical", target_size=(256,256), batch_size=32, subset='validation', seed = 9)

Found 12000 validated image filenames belonging to 16 classes.


## Setting batch size and training and validation steps

In [16]:
batch_size = 32
epochs = 50
train_steps = train_data.samples // batch_size
val_steps = validation_data.samples // batch_size

## Model 1

In [17]:
# Use VGG-16 pretrained network without Fully Connected layers and initilize all the weights with Imagenet trained weights. 
# After VGG-16 network without FC layers, add a new Conv block ( 1 Conv layer and 1 Maxpooling ), 2 FC layers and an output layer to classify 16 classes.
# Final architecture will be INPUT --> VGG-16 without Top layers(FC) --> Conv Layer --> Maxpool Layer --> 2 FC layers --> Output Layer.
# Training only new Conv block, FC layers, output layer.
def model_1(input_shape = (256,256,3), n_classes = 16):
    vgg = VGG16(weights='imagenet', include_top=False, input_shape= input_shape) # lodaing vgg 16 network
    for layer in vgg.layers:
        layer.trainable = False # Setting training of vgg network to false
    vgg_out = vgg.layers[-1].output
    x = Conv2D(filters=16,kernel_size=(3,3),padding="same",strides=2, activation="relu")(vgg_out)
    x = MaxPool2D(pool_size=(2,2))(x)
    x = Flatten()(x)
    x = Dense(64, activation = 'relu')(x)
    x = Dense(32, activation = 'relu')(x)
    output = Dense(n_classes, activation = 'softmax')(x)
    model = Model(inputs=vgg.input, outputs=output)
    return model

  


In [18]:
model_1 = model_1(input_shape = (256,256,3), n_classes = 16)
print(model_1.summary())

58900480/58889256 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                              

In [19]:
model_1.compile(loss = "categorical_crossentropy", optimizer ='Adam', metrics=["accuracy"])

In [20]:
history = model_1.fit(train_data,steps_per_epoch=train_steps, epochs=epochs,
                              validation_data=validation_data,validation_steps=val_steps)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/50
1125/1125 [==============================] - 558s 483ms/step - loss: 1.5355 - accuracy: 0.5240 - val_loss: 1.2240 - val_accuracy: 0.6237
Epoch 2/50
1125/1125 [==============================] - 538s 478ms/step - loss: 1.1384 - accuracy: 0.6499 - val_loss: 1.1499 - val_accuracy: 0.6475
Epoch 3/50
1125/1125 [==============================] - 538s 478ms/step - loss: 1.0154 - accuracy: 0.6867 - val_loss: 1.0694 - val_accuracy: 0.6818
Epoch 4/50
1125/1125 [==============================] - 538s 478ms/step - loss: 0.9279 - accuracy: 0.7126 - val_loss: 1.0670 - val_accuracy: 0.6775
Epoch 5/50
1125/1125 [==============================] - 538s 478ms/step - loss: 0.8682 - accuracy: 0.7331 - val_loss: 1.0769 - val_accuracy: 0.6828
Epoch 6/50
1125/1125 [==============================] - 538s 478ms/step - loss: 0.8188 - accuracy: 0.7480 - val_loss: 1.0610 - val_accuracy: 0.6872
Epoch 7/50
1125/1125 [==============================] - 538s 478ms/step - loss: 0.7672 - accuracy: 0.7627 - val_

KeyboardInterrupt: ignored

## Plotting accuracy

In [ ]:
loss_train = history.history['acc']
loss_val = history.history['val_acc']
epochs = range(epochs)
plt.plot(epochs, loss_train, 'g', label='Training accuracy')
plt.plot(epochs, loss_val, 'b', label='validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title("Accuracy of Model 1")
plt.show()

## Plotting model 1 architecture

In [ ]:
tf.keras.utils.plot_model(model_1, to_file='model_1.png')

## Model 2

In [ ]:
#  After VGG-16 network without FC layers, use conv layers only as Fully connected layer.Any FClayer can be converted to a CONV layer. 
# This conversion will reduce the No of Trainable parameters in FC layers. Train only last 2 Conv layers identical to FC layers, 1 output layer.
def model_2(input_shape = (256,256,3), n_classes = 16):
  # X_input = Input(input_shape)
    vgg = VGG16(weights='imagenet', include_top=False, input_shape= input_shape)
    for layer in vgg.layers:
        layer.trainable = False
    vgg_out = vgg.layers[-1].output
    x = Conv2D(filters=4096,kernel_size=8,padding="same",strides=1, activation="relu")(vgg_out)
    x = Conv2D(filters=4096,kernel_size=1,padding="same",strides=1, activation="relu")(x)
    x = MaxPool2D(pool_size=(2,2))(x)
    x = Flatten()(x)
    output = Dense(n_classes, activation = 'softmax')(x)
    model = Model(inputs=vgg.input, outputs=output)
    return model

In [ ]:
model_2 = model_2(input_shape = (256,256,3), n_classes = 16)
print(model_2.summary())

In [ ]:
model_2.compile(loss = "categorical_crossentropy", optimizer ='Adam', metrics=["accuracy"])
history = model_2.fit(train_data,steps_per_epoch=train_steps, epochs=epochs,
                              validation_data=validation_data,validation_steps=val_steps)

## Plotting Accuracy curve of Model 2

In [ ]:
loss_train = history.history['acc']
loss_val = history.history['val_acc']
epochs = range(epochs)
plt.plot(epochs, loss_train, 'g', label='Training accuracy')
plt.plot(epochs, loss_val, 'b', label='validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title("Accuracy of Model 2")
plt.show()

## Plotting Model 2 architecture

In [ ]:
tf.keras.utils.plot_model(model_2, to_file='model_2.png')

## Model 3

In [ ]:
# Use same network as Model-2 'INPUT --> VGG-16 without Top layers(FC) --> 2 Conv Layers identical to FC --> Output Layer' 
# and train only Last 6 Layers of VGG-16 network, 2 Conv layers identical to FC layers, 1 output layer.
def model_3(input_shape = (256,256,3), n_classes = 16):
  # X_input = Input(input_shape)
    vgg = VGG16(weights='imagenet', include_top=False, input_shape= input_shape)
    for layer in vgg.layers[-6:]:
        layer.trainable = True
    vgg_out = vgg.layers[-1].output
    x = Conv2D(filters=4096,kernel_size=8,padding="same",strides=1, activation="relu")(vgg_out)
    x = Conv2D(filters=4096,kernel_size=1,padding="same",strides=1, activation="relu")(x)
    x = MaxPool2D(pool_size=(2,2))(x)
    x = Flatten()(x)
    output = Dense(n_classes, activation = 'softmax')(x)
    model = Model(inputs=vgg.input, outputs=output)
    return model

In [ ]:
model_3 = model_3(input_shape = (256,256,3), n_classes = 16)
print(model_3.summary())

In [ ]:
model_3.compile(loss = "categorical_crossentropy", optimizer ='Adam', metrics=["accuracy"])
history = model_3.fit(train_data,steps_per_epoch=train_steps, epochs=epochs,
                              validation_data=validation_data,validation_steps=val_steps)

## Plotting Accuracy of Model 3

In [ ]:
loss_train = history.history['acc']
loss_val = history.history['val_acc']
epochs = range(epochs)
plt.plot(epochs, loss_train, 'g', label='Training accuracy')
plt.plot(epochs, loss_val, 'b', label='validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title("Accuracy of Model 3")
plt.show()

## Plotting Model 3 Architure

In [ ]:
tf.keras.utils.plot_model(model_3, to_file='model_3.png')